<a href="https://colab.research.google.com/github/martin-fabbri/colab-notebooks/blob/master/deeplearning.ai/tf/c3_w2_loading_text.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Load Text

We will use lower-level utilities like `tf.data.TextLineDataset` to load text files, and `tf.text` to preprocess the data for finer-grain control.

In [1]:
!pip install tensorflow_text -q

     |████████████████████████████████| 2.6MB 12.3MB/s 


In [2]:
import collections
import pathlib
import re
import string

import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_text as tf_text

from tensorflow.keras import layers
from tensorflow.keras import losses
from tensorflow.keras import preprocessing
from tensorflow.keras import utils
from tensorflow.keras import utils
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

tf.__version__, tfds.__version__

('2.3.0', '4.0.1')

## Predict the tag for a Stack Overflow question

In [3]:
data_url = 'https://storage.googleapis.com/download.tensorflow.org/data/stack_overflow_16k.tar.gz'
dataset = utils.get_file(
    'stack_overflow_16k.tar.gz',
    data_url,
    untar=True,
    cache_dir='stack_overflow',
    cache_subdir=''
)
dataset_dir = pathlib.Path(dataset).parent
dataset_dir, pathlib.Path(dataset)

6053888/6053168 [==============================] - 0s 0us/step


(PosixPath('/tmp/.keras'), PosixPath('/tmp/.keras/stack_overflow_16k.tar.gz'))

In [4]:
list(dataset_dir.iterdir())

[PosixPath('/tmp/.keras/train'),
 PosixPath('/tmp/.keras/README.md'),
 PosixPath('/tmp/.keras/test'),
 PosixPath('/tmp/.keras/stack_overflow_16k.tar.gz.tar.gz')]

In [5]:
train_dir = dataset_dir/'train'
list(train_dir.iterdir())

[PosixPath('/tmp/.keras/train/python'),
 PosixPath('/tmp/.keras/train/csharp'),
 PosixPath('/tmp/.keras/train/javascript'),
 PosixPath('/tmp/.keras/train/java')]

The `train/csharp`, `train/java`, `train/python`, and `train/javascript` directories contain many text files, each of wich is Stack Overflow question. Print a file and inspect the data.

In [6]:
sample_file = train_dir/'python/1757.txt'
with open(sample_file) as f:
  print(f.read())

"i want to return a specific list when i have a collection of lists i want to return a specific list, when i have a collection of lists, but i am not to sure how to do this. i tried this approach but it didn't work. any ideas..this_list1 = [2,3,4,5].this_list2 = [5,6,9,8]..x = input(""which list do you want"")..print this_list(x)"



## Load the dataset

Next, we will load the data off disk and prepare it into a format suitable for training. To do so, you will use `text_dataset_from_directory` utility to create a labeled `tf.data.Dataset`.

The `preprocessing.text_dataset_from_directory` expects a directory structure as follows.

```
train/
...csharp/
......1.txt
......2.txt
...java/
......1.txt
......2.txt
...javascript/
......1.txt
......2.txt
...python/
......1.txt
......2.txt


When running a machine learning experiment, it is a best practice to divide your dataset into three splits: train, validation and test. Test Stack Overflow has already been divided into train and test, but it lack a validation set. Create a validation set using a 80:20 split of the training data by using the `validation_split` argument below.


In [7]:
batch_size = 32
seed = 42

raw_train_ds = preprocessing.text_dataset_from_directory(
    train_dir,
    batch_size=batch_size,
    validation_split=0.2,
    subset='training',
    seed=seed    
)

Found 8000 files belonging to 4 classes.
Using 6400 files for training.


As you can see above, there are 8,000 examples in the training folder, of which you will use 80% (or 6,400) for training. As you will see in a moment, you can train a model by passing a `tf.data.Dataset` directly to `model.fit`. First, iterate over the dataset and print out a few examples, to get a feel for the data.

Note: To increase the difficulty of the classification problem, the dataset author replaced occurrences of the words *Python*, *CSharp*, *JavaScript*, or *Java* in the programming question with the word *blank*.

In [8]:
for text_batch, label_batch in raw_train_ds.take(1):
  for i in range(10):
    print('Question:', text_batch.numpy()[i][:100], '...')
    print('Label:', label_batch.numpy()[i])

Question: b'"my tester is going to the wrong constructor i am new to programming so if i ask a question that can' ...
Label: 1
Question: b'"blank code slow skin detection this code changes the color space to lab and using a threshold finds' ...
Label: 3
Question: b'"option and validation in blank i want to add a new option on my system where i want to add two text' ...
Label: 1
Question: b'"exception: dynamic sql generation for the updatecommand is not supported against a selectcommand th' ...
Label: 0
Question: b'"parameter with question mark and super in blank, i\'ve come across a method that is formatted like t' ...
Label: 1
Question: b'call two objects wsdl the first time i got a very strange wsdl. ..i would like to call the object (i' ...
Label: 0
Question: b'how to correctly make the icon for systemtray in blank using icon sizes of any dimension for systemt' ...
Label: 0
Question: b'"is there a way to check a variable that exists in a different script than the original one? i\'m 

The labels are `0`, `1`, `2` or `3`. To see which of these correspond to which string label, you can check the `class_names` property on the dataset.

In [9]:
for i, label in enumerate(raw_train_ds.class_names):
  print('Label', i, 'corresponds to', label)

Label 0 corresponds to csharp
Label 1 corresponds to java
Label 2 corresponds to javascript
Label 3 corresponds to python


In [10]:
raw_train_ds.class_names

['csharp', 'java', 'javascript', 'python']

In [11]:
raw_val_ds = preprocessing.text_dataset_from_directory(
    train_dir,
    batch_size=batch_size,
    validation_split=0.2,
    subset='validation',
    seed=seed
)

Found 8000 files belonging to 4 classes.
Using 1600 files for validation.


In [12]:
test_dir = dataset_dir/'test'
raw_test_ds = preprocessing.text_dataset_from_directory(
    test_dir,
    batch_size=batch_size
)

Found 8000 files belonging to 4 classes.


Next, you will standardize, tokenize, and vectorize the data using the `preprocessing.TextVectorization` layer.
* Standardization refers to preprocessing the text, typically to remove punctuation or HTML elements to simplify the dataset.

* Tokenization refers to splitting strings into tokens (for example, splitting a sentence into individual words by splitting on whitespace).

* Vectorization refers to converting tokens into numbers so they can be fed into a neural network.

All of these tasks can be accomplished with this layer. You can learn more about each of these in the [API doc](https://www.tensorflow.org/api_docs/python/tf/keras/layers/experimental/preprocessing/TextVectorization).

* The default standardization converts text to lowercase and removes punctuation.

* The default tokenizer splits on whitespace.

* The default vectorization mode is `int`. This outputs integer indices (one per token). This mode can be used to build models that take word order into account. You can also use other modes, like `binary`, to build bag-of-word models.


You will build two modes to learn more about these. First, you will use the `binary` model to build a bag-of-words model. Next, you will use the `int` mode with a 1D ConvNet.

In [13]:
VOCAB_SIZE=10000

binary_vectorize_layer = TextVectorization(
    max_tokens=VOCAB_SIZE,
    output_mode='binary'
)

For `int` mode, in addition to maximum vocabulary size, you need to set an explicit maximum sequence length, which will cause the layer to pad or truncate sequences to exactly sequence_length values.

In [14]:
MAX_SEQUENCE_LENGHT = 250

int_vectorize_layer = TextVectorization(
    max_tokens=VOCAB_SIZE,
    output_mode='int',
    output_sequence_length=MAX_SEQUENCE_LENGHT
)

Next, you will call adapt to fit the state of the preprocessing layer to the dataset. This will cause the model to build an index of strings to integers.

Note: it's important to only use your training data when calling adapt (using the test set would leak information).

In [15]:
# make a text-only dataset (without labels), then call adapt
train_text = raw_train_ds.map(lambda text, labels: text)
binary_vectorize_layer.adapt(train_text)
int_vectorize_layer.adapt(train_text)

See the results of using these layers to process data:

In [16]:
def binary_vectorize_text(text, label):
  text = tf.expand_dims(text, -1)
  return binary_vectorize_layer(text), label

In [17]:
def int_vectorize_text(text, label):
  text = tf.expand_dims(text, -1)
  return int_vectorize_layer(text), label

In [18]:
# retrieve a batch (of 32 reviews and labels) from the dataset
text_batch, label_batch = next(iter(raw_train_ds))
first_question, first_label = text_batch[0], label_batch[0]
print('question:', first_question)
print('label:', first_label)

question: tf.Tensor(b'"function expected error in blank for dynamically created check box when it is clicked i want to grab the attribute value.it is working in ie 8,9,10 but not working in ie 11,chrome shows function expected error..&lt;input type=checkbox checked=\'checked\' id=\'symptomfailurecodeid\' tabindex=\'54\' style=\'cursor:pointer;\' onclick=chkclickevt(this);  failurecodeid=""1"" &gt;...function chkclickevt(obj) { .    alert(obj.attributes(""failurecodeid""));.}"\n', shape=(), dtype=string)
label: tf.Tensor(2, shape=(), dtype=int32)


In [19]:
print('binary vectorized question:', binary_vectorize_text(first_question, first_label)[0])

binary vectorized question: tf.Tensor([[1. 1. 1. ... 0. 0. 0.]], shape=(1, 10000), dtype=float32)


In [20]:
print('int vectorized question:', int_vectorize_text(first_question, first_label)[0])

int vectorized question: tf.Tensor(
[[  38  450   65    7   16   12  892  265  186  451   44   11    6  685
     3   46    4 2062    2  485    1    6  158    7  479    1   26   20
   158    7  479    1  502   38  450    1 1767 1763    1    1    1    1
     1    1    1    1    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0  

As you can see above, `binary` mode returns an array denoting which tokens exist at least once in the input, while `int` mode replaces each token by an integer, thus preserving their order. You can lookup the token (string) that each integer corresponds to by calling `.get_vocabulary()` on the layer.

In [21]:
print('1289 --->', int_vectorize_layer.get_vocabulary()[1289])
print('2200 --->', int_vectorize_layer.get_vocabulary()[2200])
print(f'Vocabulary size: {len(int_vectorize_layer.get_vocabulary())}')

1289 ---> roman
2200 ---> desktop
Vocabulary size: 10000


You are nearly ready to train your model. As a final preprocessing step, you will apply the `TextVectorization` layers you created earlier to the train, validation, and test dataset.

In [22]:
binary_train_ds = raw_train_ds.map(binary_vectorize_text)
binary_val_ds = raw_val_ds.map(binary_vectorize_text)
binary_test_ds = raw_test_ds.map(binary_vectorize_text)

int_train_ds = raw_train_ds.map(int_vectorize_text)
int_val_ds = raw_val_ds.map(int_vectorize_text)
int_test_ds = raw_test_ds.map(int_vectorize_text)


### Configure the dataset for performance

These are two important methods you should use when loading data to make sure that I/O does not become blocking.

`.cache()` keeps data in memory after it's loaded off disk. This will ensure the dataset does not become a bottleneck while training your model. If your dataset is too large to fit into memory, you can also use this method to create a performant on-disk cache, which is more efficient to read than many small files.

`.prefetch()` overlaps data preprocessing and model execution while training. 


In [23]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

def configure_dataset(dataset):
  return dataset.cache().prefetch(buffer_size=AUTOTUNE)

In [24]:
binary_train_ds = configure_dataset(binary_train_ds)
binary_val_ds = configure_dataset(binary_val_ds)
binary_test_ds = configure_dataset(binary_test_ds)

int_train_ds = configure_dataset(int_train_ds)
int_val_ds = configure_dataset(int_val_ds)
int_test_ds = configure_dataset(int_test_ds)

In [25]:
binary_model = tf.keras.Sequential([layers.Dense(4)])
binary_model.compile(
    loss=losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer='adam',
    metrics=['accuracy']
)
history = binary_model.fit(
    binary_train_ds,
    validation_data=binary_val_ds,
    epochs=10
)

Epoch 1/10
200/200 [==============================] - 3s 13ms/step - loss: 1.1125 - accuracy: 0.6473 - val_loss: 0.9102 - val_accuracy: 0.7750
Epoch 2/10
200/200 [==============================] - 1s 3ms/step - loss: 0.7756 - accuracy: 0.8197 - val_loss: 0.7479 - val_accuracy: 0.7981
Epoch 3/10
200/200 [==============================] - 1s 3ms/step - loss: 0.6253 - accuracy: 0.8616 - val_loss: 0.6629 - val_accuracy: 0.8138
Epoch 4/10
200/200 [==============================] - 1s 3ms/step - loss: 0.5324 - accuracy: 0.8858 - val_loss: 0.6098 - val_accuracy: 0.8200
Epoch 5/10
200/200 [==============================] - 1s 3ms/step - loss: 0.4667 - accuracy: 0.9045 - val_loss: 0.5734 - val_accuracy: 0.8275
Epoch 6/10
200/200 [==============================] - 1s 3ms/step - loss: 0.4167 - accuracy: 0.9170 - val_loss: 0.5468 - val_accuracy: 0.8325
Epoch 7/10
200/200 [==============================] - 1s 3ms/step - loss: 0.3766 - accuracy: 0.9266 - val_loss: 0.5268 - val_accuracy: 0.8363
Epoch

In [26]:
def create_model(vocab_size, num_labels):
  model = tf.keras.Sequential([
    layers.Embedding(vocab_size, 64, mask_zero=True),
    layers.Conv1D(64, 5, padding='valid', activation='relu', strides=2),
    layers.GlobalMaxPool1D(),
    layers.Dense(num_labels)
  ])
  return model

In [27]:
int_model = create_model(vocab_size=VOCAB_SIZE + 1, num_labels=4)
int_model.compile(
    loss=losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer='adam',
    metrics=['accuracy']
)
history = int_model.fit(int_train_ds, validation_data=int_val_ds, epochs=5)

Epoch 1/5
200/200 [==============================] - 4s 20ms/step - loss: 1.1474 - accuracy: 0.5055 - val_loss: 0.7611 - val_accuracy: 0.6931
Epoch 2/5
200/200 [==============================] - 2s 10ms/step - loss: 0.6191 - accuracy: 0.7638 - val_loss: 0.5359 - val_accuracy: 0.8056
Epoch 3/5
200/200 [==============================] - 2s 10ms/step - loss: 0.3695 - accuracy: 0.8823 - val_loss: 0.4769 - val_accuracy: 0.8256
Epoch 4/5
200/200 [==============================] - 2s 10ms/step - loss: 0.2051 - accuracy: 0.9527 - val_loss: 0.4787 - val_accuracy: 0.8238
Epoch 5/5
200/200 [==============================] - 2s 10ms/step - loss: 0.1024 - accuracy: 0.9817 - val_loss: 0.5095 - val_accuracy: 0.8188


In [28]:
print('Linear model on binary vectorized data:')
print(binary_model.summary())

Linear model on binary vectorized data:
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 4)                 40004     
Total params: 40,004
Trainable params: 40,004
Non-trainable params: 0
_________________________________________________________________
None


In [29]:
print('ConvNet model on int vectorized data:')
print(int_model.summary())

ConvNet model on int vectorized data:
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 64)          640064    
_________________________________________________________________
conv1d (Conv1D)              (None, None, 64)          20544     
_________________________________________________________________
global_max_pooling1d (Global (None, 64)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 4)                 260       
Total params: 660,868
Trainable params: 660,868
Non-trainable params: 0
_________________________________________________________________
None


In [30]:
binary_loss, binary_accuracy = binary_model.evaluate(binary_test_ds)
int_loss, int_accuracy = int_model.evaluate(int_test_ds)

print(f'Binary model accuracy: {binary_accuracy:2.2%}')
print(f'Int model accuracy: {int_accuracy:2.2%}')

250/250 [==============================] - 3s 10ms/step - loss: 0.5176 - accuracy: 0.8115
Binary model accuracy: 81.55%
Int model accuracy: 81.15%


### Export the model

In the code above, you applied the `TextVectorization` layer to the dataset before feeding text to the model. If you want to make your model capable of processing raw strings (for example, to simplify deploying it), you can include the `TextVectorization` layer inside your model. To do so, you can create a new model using the weights you just trained.

In [31]:
export_model = tf.keras.Sequential([
  binary_vectorize_layer,
  binary_model,
  layers.Activation('sigmoid')
])

export_model.compile(
    loss=losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer='adam',
    metrics=['accuracy']
)

#export_model.build()
#export_model.summary()

Now your model can take raw strings as input and predict a score for each label using model.predict. Define a function to find the label with the maximum score:

In [32]:
def get_string_labels(predicted_scores_batch):
  predicted_int_labels = tf.argmax(predicted_scores_batch, axis=1)
  predicted_labels = tf.gather(
      raw_train_ds.class_names, 
      predicted_int_labels
  )
  return predicted_labels

Run inference on new data

In [33]:
inputs = [
  "how do I extract keys from a dict into a list?",  # python
  "debug public static void main(string[] args) {...}",  # java
]

predicted_scores = export_model.predict(inputs)
predicted_labels = get_string_labels(predicted_scores)
for input, label in zip(inputs, predicted_labels):
  print("Question: ", input)
  print("Predicted label: ", label.numpy())

Question:  how do I extract keys from a dict into a list?
Predicted label:  b'python'
Question:  debug public static void main(string[] args) {...}
Predicted label:  b'java'


Including the text preprocessing logic inside your model enables you to export a model for production that simplifies deployment, and reduces the potential for [train/test skew](https://developers.google.com/machine-learning/guides/rules-of-ml#training-serving_skew).

> Rule #32: Re-use code between your training pipeline and your serving pipeline whenever possible.

There is a performance difference to keep in mind when choosing where to apply your `TextVectorization` layer. Using it outside of your model enables you to do asynchronous CPU processing and buffering of your data when training on GPU. So, if you're training your model on the GPU, you probably want to go with this option to get the best performance while developing your model, then switch to including the TextVectorization layer inside your model when you're ready to prepare for deployment.

> Using `TextVectorization` outside of your model enables you to do asynchronous CPU processing and buffering of your data when training on GPU.

## Predict the author of Illiad translations 

The following provides an example of using `tf.data.TextLineDataset` to load examples from text files, and `tf.text` to preprocess the data. In this example, you will use three different English translations of the same work, Homer's Illiad, and train a model to identify the translator given a single line of text.

### Download and explore the dataset

The texts of the three translations are by:

 - [William Cowper](https://en.wikipedia.org/wiki/William_Cowper) — [text](https://storage.googleapis.com/download.tensorflow.org/data/illiad/cowper.txt)

 - [Edward, Earl of Derby](https://en.wikipedia.org/wiki/Edward_Smith-Stanley,_14th_Earl_of_Derby) — [text](https://storage.googleapis.com/download.tensorflow.org/data/illiad/derby.txt)

- [Samuel Butler](https://en.wikipedia.org/wiki/Samuel_Butler_%28novelist%29) — [text](https://storage.googleapis.com/download.tensorflow.org/data/illiad/butler.txt)

The text files used in this tutorial have undergone some typical preprocessing tasks like removing document header and footer, line numbers and chapter titles. Download these lightly munged files locally.

In [34]:
DIRECTORY_URL = 'https://storage.googleapis.com/download.tensorflow.org/data/illiad/'
FILE_NAMES = ['cowper.txt', 'derby.txt', 'butler.txt']

for name in FILE_NAMES:
  text_dir = utils.get_file(name, origin=DIRECTORY_URL + name)

parent_dir = pathlib.Path(text_dir).parent
list(parent_dir.iterdir())

811008/807992 [==============================] - 0s 0us/step


[PosixPath('/root/.keras/datasets/derby.txt'),
 PosixPath('/root/.keras/datasets/cowper.txt'),
 PosixPath('/root/.keras/datasets/butler.txt')]

### Load the dataset

You will use `TextLineDataset`, which is designed to create a `tf.data.Dataset` from a text file in which each example is a line of text from the original file, whereas `text_dataset_from_directory` treats all contents of a file as a single example. `TextLineDataset` is useful for text data that is primarily line-based (for example, poetry or error logs). 

> `TextLineDataset` reads each sample from a line of text from the original file, whereas `text_dataset_from_directory` treats all contents of as a single sample.

Iterate through these files, loading each one into its own dataset. Each example needs to be individually labeled, so use `tf.data.Dataset.map` to apply a labeler function to each one. This will iterate over every example in the dataset, returning (`example, label`) pairs.

In [35]:
def labeler(example, index):
  return example, tf.cast(index, tf.int64)

In [36]:
labeled_data_sets = []

for i, file_name in enumerate(FILE_NAMES):
  lines_dataset = tf.data.TextLineDataset(str(parent_dir/file_name))
  labeled_dataset = lines_dataset.map(lambda ex: labeler(ex, i))
  labeled_data_sets.append(labeled_dataset)

labeled_data_sets[:3]

[<MapDataset shapes: ((), ()), types: (tf.string, tf.int64)>,
 <MapDataset shapes: ((), ()), types: (tf.string, tf.int64)>,
 <MapDataset shapes: ((), ()), types: (tf.string, tf.int64)>]

Next, you'll combine these labeled datasets into a single dataset, and shuffle it.

In [37]:
BUFFER_SIZE = 50000
BATCH_SIZE = 64
VALIDATION_SIZE = 5000

In [55]:
all_labeled_data = labeled_data_sets[0]
for labeled_dataset in labeled_data_sets[1:]:
  all_labeled_data = all_labeled_data.concatenate(labeled_dataset)

all_labeled_data = all_labeled_data.shuffle(
    BUFFER_SIZE, 
    reshuffle_each_iteration=False
)

tf.data.experimental.cardinality(all_labeled_data)

<tf.Tensor: shape=(), dtype=int64, numpy=-2>

Print out a few examples as before. The dataset hasn't been batched yet, hence each entry in all_labeled_data corresponds to one data point

In [57]:
for text, label in all_labeled_data.take(10):
  print('Sentence:', text.numpy())
  print('Label:', label.numpy())

Sentence: b'Precaution, that the prize escape thee not.'
Label: 0
Sentence: b'fell into a sweet slumber. Presently they who were about the son of'
Label: 2
Sentence: b"AEneas, noble-soul'd Anchises' son,"
Label: 1
Sentence: b'So, by Achilles driven, a mingled throng'
Label: 0
Sentence: b'For of all cities of the sons of men,'
Label: 0
Sentence: b'Why, my brave horses, why be left behind?'
Label: 1
Sentence: b'Nor yet for Priam, nor for all the brave'
Label: 0
Sentence: b'There lay the rich-wrought belt the old man wore,'
Label: 1
Sentence: b'Apart from all our comrades, shall we two,'
Label: 1
Sentence: b'Sarpedon with the noble Glaucus led'
Label: 0


In [63]:
text, label = next(all_labeled_data.as_numpy_iterator())
text, label

(b'Precaution, that the prize escape thee not.', 0)

In [50]:
text, label = next(iter(all_labeled_data.take(1)))
text.numpy(), label.numpy()

(b'Turning the goal, but see them now no more;', 0)

### Prepare the dataset for training

Instead of using the Keras `TextVectorization` layer to preprocess our text dataset, you will now use the [`tf.text` API](https://www.tensorflow.org/tutorials/tensorflow_text/intro) to standardize and tokenize the data, build a vocabulary and use `StaticVocabularyTable` to map tokens to integers to feed to the model.

While tf.text provides various tokenizers, you will use the `UnicodeScriptTokenizer` to tokenize our dataset. Define a function to convert the text to lower-case and tokenize it. You will use `tf.data.Dataset.map` to apply the tokenization to the dataset.

In [58]:
tokenizer = tf_text.UnicodeScriptTokenizer()

In [59]:
def tokenize(text, unused_label):
  lower_case = tf_text.case_fold_utf8(text)
  return tokenizer.tokenize(lower_case)

In [60]:
tokenized_ds = all_labeled_data.map(tokenize)

Instructions for updating:
`tf.batch_gather` is deprecated, please use `tf.gather` with `batch_dims=-1` instead.


Next, you will build a vocabulary by sorting tokens by frequency and keeping the top `VOCAB_SIZE` tokens.

In [ ]:
tokenized_ds = configure_dataset(tokenized_ds)

vocab_dict = collections.defaultdict(lambda: 0)
for toks in tokenized_ds.as_numpy_iterator():
  